<a href="https://colab.research.google.com/github/Tom-Jung/IO_with_python/blob/main/Ch6_2.%20%EC%82%B0%EC%B6%9C%20%EB%B3%80%EB%8F%99%EC%9A%94%EC%9D%B8%20%EB%B6%84%ED%95%B4_3%EB%B6%80%EB%AC%B8%EC%9C%BC%EB%A1%9C%20%EC%8B%9C%EC%9E%91.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np   # Numpy import
import pandas as pd  # Pandas import

### 산출 변동요인분해

In [2]:
# 2015년 거래표 다운로드

url_total = "https://raw.githubusercontent.com/Tom-Jung/IO_with_python/main/data/Total_Matrix_2015_cst.csv"  # 불변 2015년 총거래표
Tot_cst_15 = pd.read_csv(url_total)

url_imp = "https://raw.githubusercontent.com/Tom-Jung/IO_with_python/main/data/Imported_Matrix_2015_cst.csv"  # 불변 2015년 수입거래표
Imp_cst_15 = pd.read_csv(url_imp)

url_dom = "https://raw.githubusercontent.com/Tom-Jung/IO_with_python/main/data/Domestic_Matrix_2015_cst.csv"  # 불변 2015년 국산거래표
Dom_cst_15 = pd.read_csv(url_dom)

In [3]:
# 2010년 거래표 다운로드

url_total = "https://raw.githubusercontent.com/Tom-Jung/IO_with_python/main/data/Total_Matrix_2010_cst.csv"  # 불변 2010년 총거래표
Tot_cst_10 = pd.read_csv(url_total)

url_imp = "https://raw.githubusercontent.com/Tom-Jung/IO_with_python/main/data/Imported_Matrix_2010_cst.csv"  # 불변 2010년 수입거래표
Imp_cst_10 = pd.read_csv(url_imp)

url_dom = "https://raw.githubusercontent.com/Tom-Jung/IO_with_python/main/data/Domestic_Matrix_2010_cst.csv"  # 불변 2010년 국산거래표
Dom_cst_10 = pd.read_csv(url_dom)

In [4]:
Tot_cst_10.shape

(41, 47)

부문통합하여 3부문 IO표 만들기

In [5]:
cdd = "https://raw.githubusercontent.com/Tom-Jung/IO_with_python/main/data/code.csv"  # 코드매치 파일
ccdd = pd.read_csv(cdd)
cd_for_itr = ccdd.to_numpy().reshape(33,2)
cd_3 = cd_for_itr[:,1]
cd_33 = cd_for_itr[:,0]
print('3부문 코드 : \n', cd_3)
print('33부문 코드 : \n', cd_33)

3부문 코드 : 
 [1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3]
33부문 코드 : 
 [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33]


In [6]:
# Mtx = np.zeros((3,32)) # 부문통합을 위한 코드가 많을 경우 상기의 코드를 수작업으로 미리 하는 것이 쉽자 않으므로 반복문을 통해 3x33 임시코드 만들기
# row=cd_3[:-2]
# col=cd_33[:-2]
# for i, j in zip(row,col):
#   Mtx[i-1,j-1] = 1
# print(Mtx)

In [7]:
code = np.array([[1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],  # 행렬 곱을 이용한 부문 통합을 위해 0과 1로 구성된 임시의 3x33 매트릭스
       [0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
       [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]])
code = code.astype(float)

In [8]:
SS_t0 = (Tot_cst_10.iloc[1:41,2:].to_numpy().astype(float)) # 10년 총거래표 numpy로 숫자만 불러오기(시간 t-1)
Im_t0 = (Imp_cst_10.iloc[1:38,2:].to_numpy().astype(float)) # 10년 수입거래표 numpy로 숫자만 불러오기
Dm_t0 = (Dom_cst_10.iloc[1:38,2:].to_numpy().astype(float)) # 10년 국산거래표 numpy로 숫자만 불러오기

SS_t5 = (Tot_cst_15.iloc[1:41,2:].to_numpy().astype(float)) # 15년 총거래표 numpy로 숫자만 불러오기(시간 t)
Im_t5 = (Imp_cst_15.iloc[1:38,2:].to_numpy().astype(float)) # 15년 수입거래표 numpy로 숫자만 불러오기
Dm_t5 = (Dom_cst_15.iloc[1:38,2:].to_numpy().astype(float)) # 10년 국산거래표 numpy로 숫자만 불러오기

In [9]:
nae_tot_row_t0 = code@SS_t0[0:32,0:32] # 중간투입(수요)의 행을 3부문으로 만들기
nae_tot_row_t5 = code@SS_t5[0:32,0:32] # 중간투입(수요)의 행을 3부문으로 만들기

In [10]:
nae_tot_col_t0 = nae_tot_row_t0@code.T  # 중간투입(수요)의 열을 3부문으로 만들기
nae_tot_col_t5 = nae_tot_row_t5@code.T  # 중간투입(수요)의 열을 3부문으로 만들기

In [11]:
print(nae_tot_col_t5)

[[3.22528700e+06 3.90574760e+07 1.18970520e+07]
 [1.77506690e+07 9.25393036e+08 3.50855135e+08]
 [7.13742400e+06 2.61663136e+08 5.75308016e+08]]


In [12]:
va_row_t0 = SS_t0[32:42,0:32]@code.T # 중투합, 피용자보수, 영업잉여, 고정자본소모, 기타생산세, 조정항, 부가가치계, 산출액(8X3)
va_row_t5 = SS_t5[32:41,0:32]@code.T # 중투합, 피용자보수, 영업잉여, 고정자본소모, 기타생산세, 조정항, 부가가치계, 산출액(7X3)
va_row_t5

array([[2.81133800e+07, 1.22611365e+09, 9.38060211e+08],
       [5.24346400e+06, 1.76814973e+08, 5.68154608e+08],
       [2.25764420e+07, 1.39966930e+08, 2.78921797e+08],
       [4.82839300e+06, 1.14820331e+08, 1.90020957e+08],
       [6.10044000e+05, 5.34377850e+07, 8.20549440e+07],
       [3.32583430e+07, 4.85040019e+08, 1.11915231e+09],
       [6.13717230e+07, 1.71115367e+09, 2.05721252e+09]])

In [13]:
fd_cd = np.array([[1,1,0,0,0,0,0,0,0],   # 최종수요를 통합하기 위한 0과 1로된 행렬
                  [0,0,1,1,1,1,0,0,0],
                  [0,0,0,0,0,0,1,0,0],
                  [0,0,0,0,0,0,0,1,0],
                  [0,0,0,0,0,0,0,0,1]])


In [14]:
fd_t0 = SS_t0[0:32, 33:42]
fd_t5 = SS_t5[0:32, 33:42]

In [15]:
fd_row_t0=code@fd_t0@fd_cd.T
fd_row_t5=code@fd_t5@fd_cd.T
fd_row_t0

array([[1.60096530e+07, 2.64591100e+06, 8.29846000e+05, 1.94854100e+07,
        6.89037430e+07],
       [1.66514215e+08, 1.37440491e+08, 4.56482521e+08, 7.60437227e+08,
        1.84778830e+09],
       [7.40290338e+08, 3.10398421e+08, 7.85462050e+07, 1.12923496e+09,
        1.81932431e+09]])

In [16]:
fd_row_sm_t0 = np.sum((fd_row_t0), axis=0) # 최종수요의 행합
fd_row_sm_t5 = np.sum((fd_row_t5), axis=0) # 최종수요의 행합

In [17]:
fd_sm_t0=fd_row_sm_t0.reshape(1,5)
fd_sm_t5=fd_row_sm_t5.reshape(1,5)

In [18]:
nae_va_ct_t0 = np.concatenate((nae_tot_col_t0, va_row_t0)) # 중간투입(수요)와 부가가치 및 산출액 합치기
nae_va_ct_t5 = np.concatenate((nae_tot_col_t5, va_row_t5)) # 중간투입(수요)와 부가가치 및 산출액 합치기

In [19]:
nae_va_ct_sm_t0 = nae_va_ct_t0.sum(axis=1).reshape(11,1)  # 부가가치 및 산출액의 열합
nae_va_ct_sm_t5 = nae_va_ct_t5.sum(axis=1).reshape(10,1)  # 부가가치 및 산출액의 열합

In [20]:
nnull_t0 = np.zeros((7,5)) # 0행렬
nnull_t5 = np.zeros((6,5)) # 0행렬

In [21]:
fd_nnull_t0 = np.concatenate((fd_row_t0, fd_sm_t0, nnull_t0),axis=0)  # 최종수요, 최종수요 행합 및 0행렬 합치기
fd_nnull_t5 = np.concatenate((fd_row_t5, fd_sm_t5, nnull_t5),axis=0)  # 최종수요, 최종수요 행합 및 0행렬 합치기

In [22]:
tot_f_t0 = np.concatenate((nae_va_ct_t0,nae_va_ct_sm_t0, fd_nnull_t0), axis=1)/1000000  # 최종 3x3 부문통합된 거래표
tot_f_t5 = np.concatenate((nae_va_ct_t5,nae_va_ct_sm_t5, fd_nnull_t5), axis=1)/1000000  # 최종 3x3 부문통합된 거래표

In [23]:
tot_F_t0 = pd.DataFrame(tot_f_t0.round(3), columns = ['농산품', '공산품', '서비스', '중간수요계', '소비','투자', '수출', '최종수요계', '총수요계'],
                     index=['농산품', '공산품', '서비스', '중간투입계', '피용자보수','영업잉여', '고정자본소모', '생산세(보조금공제)','조정항', '부가가치계', '총투입액']) # 변수명 붙여서 pandas로 내보내기


In [24]:
tot_F_t5 = pd.DataFrame(tot_f_t5.round(3), columns = ['농산품', '공산품', '서비스', '중간수요계', '소비','투자', '수출', '최종수요계', '총수요계'],
                     index=['농산품', '공산품', '서비스', '중간투입계', '피용자보수','영업잉여', '고정자본소모', '생산세(보조금공제)','부가가치계', '총투입액']) # 변수명 붙여서 pandas로 내보내기


In [25]:
# 2010, 2015년 3부문 불변표 내보내기
tot_F_t0.to_csv('10년_총거래표_3부문_불변.csv', encoding="utf-8-sig")
tot_F_t5.to_csv('15년_총거래표_3부문_불변.csv', encoding="utf-8-sig")

### 수입거래표 3부문 통합

In [26]:
nae_imp_row_t0 = code@Im_t0[0:32,0:32] # 중간투입(수요)의 행을 3부문으로 만들기
nae_imp_row_t5 = code@Im_t5[0:32,0:32] # 중간투입(수요)의 행을 3부문으로 만들기

In [27]:
nae_imp_col_t0 = nae_imp_row_t0@code.T  # 중간투입(수요)의 열을 3부문으로 만들기
nae_imp_col_t5 = nae_imp_row_t5@code.T  # 중간투입(수요)의 열을 3부문으로 만들기

In [28]:
Im_t0.shape

(33, 42)

In [29]:
ism_row_t0 = (Im_t0[32,0:32]@code.T).reshape(1,3) # 중투합
ism_row_t5 = (Im_t5[32,0:32]@code.T).reshape(1,3) # 중투합
ism_row_t5

array([[1.88064500e+06, 3.52026400e+08, 1.23335452e+08]])

In [30]:
ifd_t0 = Im_t0[0:32, 33:42] #수입최종수요
ifd_t5 = Im_t5[0:32, 33:42] #수입최종수요

In [31]:
ifd_row_t0=code@ifd_t0@fd_cd.T
ifd_row_t5=code@ifd_t5@fd_cd.T

In [32]:
ifd_row_sm_t0 = np.sum((ifd_row_t0), axis=0) # 최종수요의 행합
ifd_row_sm_t5 = np.sum((ifd_row_t5), axis=0) # 최종수요의 행합

In [33]:
ifd_sm_t0=ifd_row_sm_t0.reshape(1,5)
ifd_sm_t5=ifd_row_sm_t5.reshape(1,5)

In [34]:
ism_row_t0.ndim

2

In [35]:
nae_im_t0 = np.concatenate((nae_imp_col_t0, ism_row_t0)) # 중간투입(수요)와 부가가치 및 산출액 합치기
nae_im_t5 = np.concatenate((nae_imp_col_t5, ism_row_t5)) # 중간투입(수요)와 부가가치 및 산출액 합치기

In [36]:
nae_col_sm_imp_t0 = nae_im_t0.sum(axis=1).reshape(4,1)  # 중간투입(수요)와 행합
nae_col_sm_imp_t5 = nae_im_t5.sum(axis=1).reshape(4,1)  # 중간투입(수요)와 행합

In [37]:
nae_im_sm_t0 = nae_col_sm_imp_t0.sum(axis=1).reshape(4,1)  # 중간투입 및 중간투입 열합
nae_im_sm_t5 = nae_col_sm_imp_t5.sum(axis=1).reshape(4,1)

In [38]:
ifd_nnull_t0 = np.concatenate((ifd_row_t0, ifd_sm_t0),axis=0)  #   최종수요 및 최종수요 행합 합치기
ifd_nnull_t5 = np.concatenate((ifd_row_t5, ifd_sm_t5),axis=0)  #   최종수요 및 최종수요 행합 합치기

In [39]:
imp_f_t0 = (np.concatenate((nae_im_t0, nae_im_sm_t0, ifd_nnull_t0), axis=1))/1000000  # 최종 3x3 부문통합된 거래표
imp_f_t5 = (np.concatenate((nae_im_t5, nae_im_sm_t5, ifd_nnull_t5), axis=1))/1000000  # 최종 3x3 부문통합된 거래표

In [40]:
imp_F_t0 = pd.DataFrame(imp_f_t0.round(3), columns = ['농산품', '공산품', '서비스', '중간수요계', '소비','투자', '수출', '최종수요계', '총수요계'],
                     index=['농산품', '공산품', '서비스', '중간투입계']) # 변수명 붙여서 pandas로 내보내기
imp_F_t5 = pd.DataFrame(imp_f_t5.round(3), columns = ['농산품', '공산품', '서비스', '중간수요계', '소비','투자', '수출', '최종수요계', '총수요계'],
                     index=['농산품', '공산품', '서비스', '중간투입계']) # 변수명 붙여서 pandas로 내보내기
imp_F_t5

,농산품,공산품,서비스,중간수요계,소비,투자,수출,최종수요계,총수요계
농산품,0.349,8.219,0.749,9.316,2.859,-0.018,0.000,2.842,12.158
공산품,1.157,327.867,76.355,405.378,53.140,53.399,2.126,108.665,514.044
서비스,0.375,15.941,46.232,62.548,28.500,6.753,0.000,35.253,97.801
중간투입계,1.881,352.026,123.335,477.242,84.500,60.134,2.126,146.760,624.002


In [41]:
# 2010, 2015년 3부문 불변표 내보내기
imp_F_t0.to_csv('10년_수입거래표_3부문_불변.csv', encoding="utf-8-sig")
imp_F_t5.to_csv('15년_수입거래표_3부문_불변.csv', encoding="utf-8-sig")

### 국산거래표 3부문 통합

In [42]:
nae_dom_row_t0 = code@Dm_t0[0:32,0:32] # 중간투입(수요)의 행을 3부문으로 만들기
nae_dom_row_t5 = code@Dm_t5[0:32,0:32] # 중간투입(수요)의 행을 3부문으로 만들기

In [43]:
nae_dom_col_t0 = nae_dom_row_t0@code.T  # 중간투입(수요)의 열을 3부문으로 만들기
nae_dom_col_t5 = nae_dom_row_t5@code.T  # 중간투입(수요)의 열을 3부문으로 만들기

In [44]:
dsm_row_t0 = (Dm_t0[32,0:32]@code.T).reshape(1,3) # 중투합
dsm_row_t5 = (Dm_t5[32,0:32]@code.T).reshape(1,3) # 중투합
dsm_row_t5

array([[2.62327350e+07, 8.74087247e+08, 8.14724759e+08]])

In [45]:
dfd_t0 = Dm_t0[0:32, 33:42] #국산최종수요
dfd_t5 = Dm_t5[0:32, 33:42] #국산최종수요

In [46]:
dfd_row_t0=code@dfd_t0@fd_cd.T
dfd_row_t5=code@dfd_t5@fd_cd.T

In [47]:
dfd_row_sm_t0 = np.sum((dfd_row_t0), axis=0) # 최종수요의 행합
dfd_row_sm_t5 = np.sum((dfd_row_t5), axis=0) # 최종수요의 행합

In [48]:
dfd_sm_t0=dfd_row_sm_t0.reshape(1,5)
dfd_sm_t5=dfd_row_sm_t5.reshape(1,5)

In [49]:
nae_dm_t0 = np.concatenate((nae_dom_col_t0, dsm_row_t0)) # 중간투입(수요)와 부가가치 및 산출액 합치기
nae_dm_t5 = np.concatenate((nae_dom_col_t5, dsm_row_t5)) # 중간투입(수요)와 부가가치 및 산출액 합치기

In [50]:
nae_col_sm_dom_t0 = nae_dm_t0.sum(axis=1).reshape(4,1)  # 중간투입(수요)와 행합
nae_col_sm_dom_t5 = nae_dm_t5.sum(axis=1).reshape(4,1)  # 중간투입(수요)와 행합

In [51]:
nae_dm_sm_t0 = nae_col_sm_dom_t0.sum(axis=1).reshape(4,1)  # 중간투입 및 중간투입 열합
nae_dm_sm_t5 = nae_col_sm_dom_t5.sum(axis=1).reshape(4,1)

In [52]:
dfd_nnull_t0 = np.concatenate((dfd_row_t0, dfd_sm_t0),axis=0)  #   최종수요 및 최종수요 행합 합치기
dfd_nnull_t5 = np.concatenate((dfd_row_t5, dfd_sm_t5),axis=0)  #   최종수요 및 최종수요 행합 합치기

In [53]:
dom_f_t0 = (np.concatenate((nae_dm_t0, nae_dm_sm_t0, dfd_nnull_t0), axis=1))/1000000  # 최종 3x3 부문통합된 거래표
dom_f_t5 = (np.concatenate((nae_dm_t5, nae_dm_sm_t5, dfd_nnull_t5), axis=1))/1000000  # 최종 3x3 부문통합된 거래표

In [54]:
dom_F_t0 = pd.DataFrame(dom_f_t0.round(3), columns = ['농산품', '공산품', '서비스', '중간수요계', '소비','투자', '수출', '최종수요계', '총수요계'],
                     index=['농산품', '공산품', '서비스', '중간투입계']) # 변수명 붙여서 pandas로 내보내기
dom_F_t5 = pd.DataFrame(dom_f_t5.round(3), columns = ['농산품', '공산품', '서비스', '중간수요계', '소비','투자', '수출', '최종수요계', '총수요계'],
                     index=['농산품', '공산품', '서비스', '중간투입계']) # 변수명 붙여서 pandas로 내보내기
dom_F_t5

,농산품,공산품,서비스,중간수요계,소비,투자,수출,최종수요계,총수요계
농산품,2.876,30.839,11.148,44.864,14.148,1.673,0.687,16.508,61.372
공산품,16.594,597.526,274.501,888.620,129.077,84.844,608.612,822.533,1711.154
서비스,6.763,245.722,529.076,781.561,827.174,342.951,105.526,1275.652,2057.213
중간투입계,26.233,874.087,814.725,1715.045,970.400,429.468,714.825,2114.693,3829.738


In [55]:
# 2010, 2015년 3부문 불변표 내보내기
dom_F_t0.to_csv('10년_국산거래표_3부문_불변.csv', encoding="utf-8-sig")
dom_F_t5.to_csv('15년_국산거래표_3부문_불변.csv', encoding="utf-8-sig")

In [56]:
S1 = tot_f_t5.astype(float)
I1 = imp_f_t5.astype(float)
S0 = tot_f_t0.astype(float)
I0 = imp_f_t0.astype(float)
S1.sum()

40931.720084

In [57]:
SS1 = np.concatenate((S1[:8,],np.zeros((1,9)), S1[8:10,] ), axis=0)  # 2015년 부가가치 조정항을 0으로 삽입(2010년과 동일한 포맷을 유지하기 위함)

In [58]:
SS0 = S0[0:3,0:3].astype(float)  # 내생항목 3x3
SS1 = S1[0:3,0:3].astype(float)
SS1.shape

(3, 3)

In [59]:
II0 = I0[0:3,0:3]  # 수입 내생항목 3x3
II1 = I1[0:3,0:3]

In [60]:
DD0 =SS0 - II0  # 국산 내생항목 3x3
DD1 =SS1 - II1
DD0

array([[  4.12948 ,  28.995364,   8.603334],
       [ 16.764262, 510.283432, 235.219162],
       [  5.354213, 209.09453 , 427.310093]])

In [61]:
TM0 = I0[0:3,8]  # 총수입벡터
TM1 = I1[0:3,8]
TM0

array([  9.798326, 416.949953,  72.426752])

In [62]:
IM0 = I0[0:3,3]  # 중간재수입벡터
IM1 = I1[0:3,3]
IM0

array([  7.690155, 325.084213,  48.330511])

In [63]:
IF0 = I0[0:3,7]  # 최종재수입벡터
IF1 = I1[0:3,7]
IF0

array([ 2.108171, 91.86574 , 24.096241])

In [64]:
SFC0 = S0[0:3,4]  # 최종수요-소비
SFC1 = S1[0:3,4]
SFC0

array([ 16.009653, 166.514215, 740.290338])

In [65]:
SFI0 = S0[0:3,5] # 최종수요-투자
SFI1 = S1[0:3,5]
SFI0

array([  2.645911, 137.440491, 310.398421])

In [66]:
SF0 = S0[0:3,7] # 최종수요합계
SF1 = S1[0:3,7]
SF0

array([  19.48541 ,  760.437227, 1129.234964])

In [67]:
EX0 = S0[0:3,6] # 수출
EX1 = S1[0:3,6]
EX0

array([  0.829846, 456.482521,  78.546205])

In [68]:
DF0 = SF0 - EX0 #소비+투자
DF1 = SF1 - EX1
DF0

array([  18.655564,  303.954706, 1050.688759])

In [69]:
SFC00 = SFC0.sum(axis=0) # 소비합계
SFC11 = SFC1.sum(axis=0)

SFI00 = SFI0.sum(axis=0) # 투자합계
SFI11 = SFI1.sum(axis=0)

EX00 = EX0.sum(axis=0) # 수출합계
EX11 = EX1.sum(axis=0)

print('2010년 소비합계 : ', SFC00)
print('2015년 소비합계 : ', SFC11)

print('2010년 투자합계 : ', SFI00)
print('2015년 투자합계 : ', SFI11)

print('2010년 수출합계 : ', EX00)
print('2015년 수출합계 : ', EX11)


2010년 소비합계 :  922.814206
2015년 소비합계 :  1054.90032
2010년 투자합계 :  450.484823
2015년 투자합계 :  489.601534
2010년 수출합계 :  535.858572
2015년 수출합계 :  716.951177


In [70]:
GDC1 = ((SFC11 - SFC00)/ SFC00) # 소비증가율
GDC2 = GDC1*SFC0  # 규모효과
GDC3 = (SFC1-SFC0) - GDC2 # 구조효과

In [71]:
GDI1 = ((SFI11 - SFI00)/ SFI00) # 투자증가율
GDI2 = GDI1*SFI0 # 규모효과
GDI3 = (SFI1-SFI0) - GDI2 # 구조효과

In [72]:
ED1 = ((EX11 - EX00)/EX00) # 수출증가율
ED2 = ED1*EX0 # 규모효과
ED3 = (EX1-EX0) - ED2 # 구조효과

In [73]:
C0 = S0[-1,0:3].T.reshape([3,1])  # 총산출액
C1 = S1[-1,0:3].T.reshape([3,1])
C0

array([[  59.105417],
       [1430.838343],
       [1746.897559]])

In [74]:
TC0 = C0.sum(axis=0)  #총산출계
TC1 = C1.sum(axis=0)
TC0

array([3236.841319])

In [75]:
V0 = S0[-1,0:3].T - S0[3,0:3].T # 부가가치계벡터 = 총투입액 - 중간투입계
V1 = S1[-1,0:3].T - S1[3,0:3].T
V1

array([  33.258343,  485.040019, 1119.152306])

In [76]:
AV0 = V0/C0 # 부가가치계수
AV1 = V1/C1
AV1

array([[5.41916397e-01, 7.90331435e+00, 1.82356344e+01],
       [1.94362106e-02, 2.83457897e-01, 6.54033783e-01],
       [1.61667026e-02, 2.35775359e-01, 5.44013949e-01]])

In [77]:
C0

array([[  59.105417],
       [1430.838343],
       [1746.897559]])

In [78]:
CT0 = np.tile(C0, reps=[1,3])   # Ct벡터를 반복해여 행렬(3x3)로 만듬
CT1 = np.tile(C1, reps=[1,3])
CT1

array([[  61.371723,   61.371723,   61.371723],
       [1711.153666, 1711.153666, 1711.153666],
       [2057.212517, 2057.212517, 2057.212517]])

In [79]:
AS0 = SS0/CT0.T # 총투입계수표
AS1 = SS1/CT1.T
# np.isnan(AS_t)

In [80]:
AD0 = DD0/CT0.T # 국산투입계수표
AD1 = DD1/CT1.T

In [81]:
II0 = AS0-AD0 # 수입투입계수표
II1 = AS1-AD1

In [82]:
M0 = IF0/DF0 # 수입최종수요/(소비+투자)
M0 = np.diag(M0)

M1 = IF1/DF1
M1 = np.diag(M1)

In [83]:
RD0 = np.linalg.inv(np.eye(3) - AD0) # 생산유발계수
RD1 = np.linalg.inv(np.eye(3) - AD1)

In [84]:
AM0 = II0/AS0 #조정항목
AM0 = AM0*AS1

AM1 = II1/AS1
AM1 = AM1*AS0

# np.nan_to_num(AM0, copy=False)
# np.nan_to_num(AM1, copy=False)

In [85]:
AA = AS1 - AS0 #투입계수변화
AS1_f = pd.DataFrame(AS1)
AS1_f.to_csv('AS1_f.csv')

#### 산출액성장요인분해
-  파쉐 및 라스파이레스 방식으로 산출액 변동을 분해하여 평균으로 성장요인을 분해
- 파쉐 방식

In [86]:
# TC1 = ((RD1@(np.eye(3)-M1))@GDC2).reshape([3,1])  # 소비규모증가효과
# TC11 = ((RD1@(np.eye(3)-M1))@GDC3).reshape([3,1]) # 소비구조증가효과
# TCTD = TC1 + TC11
# TC111 = np.concatenate((TCTD, TC1, TC11), axis=1)
# # TC111

In [87]:
# TI1 = ((RD1@(np.eye(3)-M1))@GDI2).reshape([3,1])  # 소비규모증가효과
# TI11 = ((RD1@(np.eye(3)-M1))@GDI3).reshape([3,1]) # 소비구조증가효과
# TITD = TI1 + TI11
# TI111 = np.concatenate((TITD, TI1, TI11), axis=1)

In [88]:
# TQ = TC1+TI1 # 국내최종수요규모증가효과
# TS = TC11+TI11 # 국내최종수요규조변화효과
# TQS = TQ+TS
# T1 = np.concatenate((TQS, TQ, TS), axis=1)
# T1.shape
# TQS/1000000

In [89]:
TQS=(RD1@(np.eye(3)-M1))@(DF1-DF0).reshape([3,1])
# TQS/1000000

In [90]:
TTE= RD1@(EX1-EX0).reshape([3,1]) # 수출효과
# TTE/1000000

In [91]:
# T3 = ((RD1@(M1 - M0))@DF0).reshape([3,1]) # 최종재수입대체효과
# T3 = T3*-1
# T4 = ((RD1@(II0 - II0)))@C0[0:3,] # 중간재수입대체효과
# T4 = T4*-1
# T34 = T3 + T4
# TI = np.concatenate((T34, T3, T4), axis=1)
# T4

In [92]:
T3 = ((RD1@(M0 - M1))@DF0).reshape([3,1]) # 최종재수입대체효과
T4 = ((RD1@(II0 -AM1)))@C0[0:3,] # 중간재수입대체효과
T34 = T3 + T4
TI = np.concatenate((T34, T3, T4), axis=1)
print(T3)
print(T4)

[[ -1.22838544]
 [-20.40032363]
 [-13.05506452]]
[[ -1.44484458]
 [-24.66522446]
 [ -8.5938428 ]]


In [93]:
T5 = (RD1@(AA-(II1-AM1)))@C0 # 기술변화효과
T5.shape
T5

array([[-2.65578178],
       [ 6.68340241],
       [30.74962927]])

In [94]:
DCT0 = C1-C0  #산출액 실질변동액
DCT1 = TQS + TTE + T3 + T4 + T5 # 총변동요인
DCT = DCT0-DCT1

In [95]:
SUM = np.concatenate((DCT0, DCT1, DCT, TQS, TTE, T3, T4, T5), axis=1)
SUM.shape
SUM1 = np.sum(SUM, axis=0).reshape([1,8])
PGD1015_f = (np.concatenate((SUM, SUM1), axis=0))
# PGD1015_f = (PGD1015/1000).round(1)
PGD1015_f

array([[ 2.26630600e+00,  2.26630579e+00,  2.07576365e-07,
         2.39274555e+00,  5.20257205e+00, -1.22838544e+00,
        -1.44484458e+00, -2.65578178e+00],
       [ 2.80315323e+02,  2.80315313e+02,  9.98362361e-06,
         6.17227966e+01,  2.56974662e+02, -2.04003236e+01,
        -2.46652245e+01,  6.68340241e+00],
       [ 3.10314958e+02,  3.10314955e+02,  3.30703062e-06,
         2.14444021e+02,  8.67702118e+01, -1.30550645e+01,
        -8.59384280e+00,  3.07496293e+01],
       [ 5.92896587e+02,  5.92896574e+02,  1.34982306e-05,
         2.78559563e+02,  3.48947446e+02, -3.46837736e+01,
        -3.47039118e+01,  3.47772499e+01]])

In [96]:
# Ad2 - Ad1 검증
Test = ((RD1@(AD1 - AD0)))@C0[0:3,] # 중간재수입대체효과
Test
ssum = T4+T5
print(Test)
print(ssum)
# 둘의 숫자가 동일함

[[ -4.10062636]
 [-17.98182205]
 [ 22.15578647]]
[[ -4.10062636]
 [-17.98182205]
 [ 22.15578647]]


In [97]:
PGD1015_ff = pd.DataFrame((PGD1015_f).round(3),columns = ['15-10산출액(A)', '요인별합계(B)', 'B-A', '국내최종수요증감액', '수출', '최종재수입','중간재수입', '기술변화'],

                         index=['농산품', '공산품', '서비스', '계'])  # pandas로 내보내기

In [98]:
PGD1015_ff

,15-10산출액(A),요인별합계(B),B-A,국내최종수요증감액,수출,최종재수입,중간재수입,기술변화
농산품,2.266,2.266,0.0,2.393,5.203,-1.228,-1.445,-2.656
공산품,280.315,280.315,0.0,61.723,256.975,-20.400,-24.665,6.683
서비스,310.315,310.315,0.0,214.444,86.770,-13.055,-8.594,30.750
계,592.897,592.897,0.0,278.560,348.947,-34.684,-34.704,34.777


- 라스파이레스식

In [99]:
# TC1 = ((RD0@(np.eye(3)-M0))@GDC2).reshape([3,1])  # 소비규모증가효과
# TC11 = ((RD0@(np.eye(3)-M0))@GDC3).reshape([3,1]) # 소비구조증가효과
# TCTD = TC1 + TC11
# TC111 = np.concatenate((TCTD, TC1, TC11), axis=1)
# # TC111

In [100]:
# TI1 = ((RD0@(np.eye(3)-M0))@GDI2).reshape([3,1])  # 소비규모증가효과
# TI11 = ((RD0@(np.eye(3)-M0))@GDI3).reshape([3,1]) # 소비구조증가효과
# TITD = TI1 + TI11
# TI111 = np.concatenate((TITD, TI1, TI11), axis=1)

In [101]:
# TQ = TC1+TI1 # 국내최종수요규모증가효과
# TS = TC11+TI11 # 국내최종수요규조변화효과
# TQS = TQ+TS
# T1 = np.concatenate((TQS, TQ, TS), axis=1)
# T1.shape

In [102]:
TQS=(RD0@(np.eye(3)-M0))@(DF1-DF0).reshape([3,1])
# TQS/1000000

In [103]:
T2 = (RD0@ED2).reshape([3,1]) # 수출규모증가효과
T22 = (RD0@ED3) .reshape([3,1])# 수출구조변화효과
TTE = T2+T22
T2 = np.concatenate((TTE, T2, T22), axis=1)

In [104]:
TTE= RD0@(EX1-EX0).reshape([3,1]) # 수출효과
# TTE/1000000

In [105]:
# T3 = ((RD0@(M1 - M0))@DF1).reshape([3,1]) # 최종재수입대체효과
# T3 = T3*-1
# T4 = ((RD0@(II1 - AM0)))@C1[0:3,] # 중간재수입대체효과
# T4 = T4*-1
# T34 = T3 + T4
# TI = np.concatenate((T34, T3, T4), axis=1)
# print(T3)
# print(T4)

In [106]:
T3 = ((RD0@(M0 - M1))@DF1).reshape([3,1]) # 최종재수입대체효과
T4 = ((RD0@(AM0 - II1)))@C1[0:3,] # 중간재수입대체효과
T34 = T3 + T4
TI = np.concatenate((T34, T3, T4), axis=1)
print(T3)
print(T4)

[[ -1.34334812]
 [-21.98202272]
 [-14.486877  ]]
[[ -1.70895229]
 [-29.77783028]
 [-10.21528676]]


In [107]:
T5 = (RD0@(AA-(AM0-II0)))@C1 # 기술변화효과
T5.shape
T5

array([[-3.18177534],
       [ 7.97760538],
       [35.49737509]])

In [108]:
SUM = np.concatenate((DCT0, DCT1, DCT, TQS, TTE, T3, T4, T5), axis=1)
SUM.shape
SUM1 = np.sum(SUM, axis=0).reshape([1,8])
LGD1015_f = (np.concatenate((SUM, SUM1), axis=0))
# LGD1015_f = (LGD1015/1000).round(1)
LGD1015_f

array([[ 2.26630600e+00,  2.26630579e+00,  2.07576365e-07,
         2.51688420e+00,  5.98349730e+00, -1.34334812e+00,
        -1.70895229e+00, -3.18177534e+00],
       [ 2.80315323e+02,  2.80315313e+02,  9.98362361e-06,
         6.35213201e+01,  2.60576240e+02, -2.19820227e+01,
        -2.97778303e+01,  7.97760538e+00],
       [ 3.10314958e+02,  3.10314955e+02,  3.30703062e-06,
         2.12676057e+02,  8.68436859e+01, -1.44868770e+01,
        -1.02152868e+01,  3.54973751e+01],
       [ 5.92896587e+02,  5.92896574e+02,  1.34982306e-05,
         2.78714262e+02,  3.53403424e+02, -3.78122478e+01,
        -4.17020693e+01,  4.02932051e+01]])

In [109]:
LGD1015_ff = pd.DataFrame((LGD1015_f).round(3),columns = ['15-10산출액(A)', '요인별합계(B)', 'B-A', '국내최종수요증감액', '수출', '최종재수입','중간재수입', '기술변화'],

                         index=['농산품', '공산품', '서비스', '계'])  # pandas로 내보내기

In [110]:
LGD1015_ff

,15-10산출액(A),요인별합계(B),B-A,국내최종수요증감액,수출,최종재수입,중간재수입,기술변화
농산품,2.266,2.266,0.0,2.517,5.983,-1.343,-1.709,-3.182
공산품,280.315,280.315,0.0,63.521,260.576,-21.982,-29.778,7.978
서비스,310.315,310.315,0.0,212.676,86.844,-14.487,-10.215,35.497
계,592.897,592.897,0.0,278.714,353.403,-37.812,-41.702,40.293


In [111]:
PGD1015_ff.to_csv('산출액변동요인_파쉐.csv', encoding="utf-8-sig")
LGD1015_ff.to_csv('산출액변동요인_라스파이레스.csv', encoding="utf-8-sig")

In [112]:
Ctbr_Pa =  PGD1015_f[:,3:]
Ctbr_La =  LGD1015_f[:,3:]
Ctbr_La

array([[  2.5168842 ,   5.9834973 ,  -1.34334812,  -1.70895229,
         -3.18177534],
       [ 63.52132011, 260.57624039, -21.98202272, -29.77783028,
          7.97760538],
       [212.67605747,  86.84368588, -14.486877  , -10.21528676,
         35.49737509],
       [278.71426178, 353.40342358, -37.81224784, -41.70206933,
         40.29320513]])

In [113]:
sm_gap_Pa = PGD1015_f[:,1].reshape([4,1])  # 요인별 합계
sm_gap_La = LGD1015_f[:,1].reshape([4,1])  # 요인별 합계
sm_gap_La

array([[  2.26630579],
       [280.31531302],
       [310.31495469],
       [592.8965735 ]])

In [114]:
sm_gap1_Pa = np.tile(sm_gap_Pa, reps=[1,5]) # 요인별 합계를 33x16으로 반복
sm_gap1_La = np.tile(sm_gap_La, reps=[1,5]) # 요인별 합계를 33x16으로 반복
sm_gap1_La

array([[  2.26630579,   2.26630579,   2.26630579,   2.26630579,
          2.26630579],
       [280.31531302, 280.31531302, 280.31531302, 280.31531302,
        280.31531302],
       [310.31495469, 310.31495469, 310.31495469, 310.31495469,
        310.31495469],
       [592.8965735 , 592.8965735 , 592.8965735 , 592.8965735 ,
        592.8965735 ]])

In [115]:
Ctrb_f_Pa = (Ctbr_Pa/sm_gap1_Pa)*100  # 각 항목별 기여율
Ctrb_f_La = (Ctbr_La/sm_gap1_La)*100  # 각 항목별 기여율
Ctrb_f_Pa

array([[ 105.57911286,  229.56178566,  -54.20210492,  -63.75329346,
        -117.18550013],
       [  22.01905985,   91.67342996,   -7.27763439,   -8.79909991,
           2.38424449],
       [  69.10528085,   27.961982  ,   -4.20703686,   -2.7693937 ,
           9.9091677 ],
       [  46.98282559,   58.85469095,   -5.84988599,   -5.85328258,
           5.86565203]])

In [116]:
Ctrb_ff_Pa = pd.DataFrame(Ctrb_f_Pa,columns = ['국내최종수요증감액', '수출', '최종재수입','중간재수입', '기술변화'],

                         index=['농산품', '공산품', '서비스', '계'])  # pandas로 내보내기

In [117]:
Ctrb_ff_La = pd.DataFrame(Ctrb_f_La,columns = ['국내최종수요증감액', '수출', '최종재수입','중간재수입', '기술변화'],

                         index=['농산품', '공산품', '서비스', '계'])  # pandas로 내보내기

In [118]:
Ctrb_ff_Pa.to_csv('항목별 기여율_파쉐.csv', encoding="utf-8-sig")
Ctrb_ff_La.to_csv('항목별 기여율_라스파이레스.csv', encoding="utf-8-sig")